In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
import math

In [2]:
start_date = datetime.date(2008,10,1)
end_date = datetime.date(2008,12,31) 
d = datetime.timedelta(1)

In [3]:
def get_quarter(d):
    
    quarter = "%d_Q%d" % (d.year, math.ceil(d.month/3))
    
    return quarter
    
year_quarter = get_quarter(end_date)

In [4]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

In [5]:
writer = pd.ExcelWriter('FX Rate '+year_quarter+'.xlsx')
    
for date in daterange(start_date, end_date+d):
    
    url = 'https://www.xe.com/currencytables/?from=USD&date='+str(date.year)+'-'+'{:02d}'.format(date.month)+'-'+'{:02d}'.format(date.day)

    # this is the html from the given url
    html = urlopen(url)

    soup = BeautifulSoup(html, 'html.parser')

    column_headers = [th.getText() for th in 
                      soup.findAll('tr', limit=1)[0].findAll('th')] #[0] means first row of table
    column_headers = [re.sub("[^ \w]"," ",x).strip() for x in column_headers]

    data_rows = soup.findAll('tr')[1:] #[1:] means 2nd row of table onward
    type(data_rows)  # now we have a list of table rows

    country_data = []  # create an empty list to hold all the data

    for i in range(len(data_rows)):  # for each table row
        country_row = []  # create an empty list for each country

        # for each table data element from each table row
        for td in data_rows[i].findAll('td'):        
            # get the text content and append to the country_row 
            country_row.append(td.getText())        

        # then append each country to the country_data matrix
        country_data.append(country_row)

    df = pd.DataFrame(country_data, columns=column_headers)
    
    df = df[df['Currency code'].str.len() == 3]
    df = df.drop(['Units per USD'], axis=1)
    df['Date'] = str(date.year) + '{:02d}'.format(date.month) + '{:02d}'.format(date.day)
    df = df[['Date', 'Currency code', 'Currency name', 'USD per Unit']]
    df['Date']=df['Date'].astype('int')
    df['USD per Unit']=df['USD per Unit'].astype('float')
    
    df.to_excel(writer,str(date.year) + '{:02d}'.format(date.month) + '{:02d}'.format(date.day), index=False)

writer.save()